In [1]:
from keras.models import load_model
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [2]:
modelvna = load_model('Trained Models/model-vna.h5')
modelClamp = load_model('Trained Models/model-clamp-10000.h5')

In [3]:
data=pd.DataFrame()
data=pd.read_pickle('data-vna.pkl')
datac=pd.read_pickle('data-clamp.pkl')

In [4]:
data['diet']=data.label.map(lambda x: x[5])

In [5]:
data.head()

,text,label,split,ind,diet
0,associate circle .D. Jr. 845-41-54-4 ...,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1]","[associate, circle, year, old, pound, gentlema...","[552, 1419, 9356, 5815, 6504, 3301, 7026, 1691...",1
1,MERCY care CENTER associate AR 72985 M...,"[0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0]","[mercy, care, center, associate, come, further...","[5059, 1169, 1289, 552, 1575, 3213, 2786, 4334...",0
2,Personal Overall be 81 yr\nold male present...,"[1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0]","[personal, overall, old, male, present, multip...","[6245, 5963, 5815, 4901, 6576, 5334, 8531, 286...",0
3,"HUNTINGTON EMERGENCY DEPT visit THOMAS-YOSEF,J...","[0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0]","[huntington, emergency, dept, visit, julium, v...","[3831, 2586, 2073, 9082, 4419, 9082, 1953, 612...",0
4,JENNIFER BOOKER LC Unit No NASHUA MEMORIAL HOS...,"[0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1]","[jennifer, booker, unit, nashua, memorial, hos...","[4393, 942, 8808, 5427, 5040, 3779, 144, 1953,...",0


In [6]:
datac.head()

,text,label,name,clean,diet,ind
0,\n\nRecord date: 2106-02-12\n\nCampbell Orthop...,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1]",100,"[problem, present, an, injury, to, his, left, ...",1,"[6942, 6905, 1163, 4673, 8626, 4244, 5042, 122..."
1,\n\nRecord date: 2079-05-12\n\n\n\n\n\nMERCY C...,"[0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0]",101,"[test, present, further, evaluation, problem, ...",0,"[8463, 6905, 3806, 3405, 6942, 6905, 4114, 458..."
2,\n\nRecord date: 2120-09-19\n\nPersonal Data a...,"[1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0]",102,"[treatment, present, multiple, tooth, extracti...",0,"[8743, 6905, 5723, 8655, 3469, 6942, 6905, 865..."
3,\n\nRecord date: 2067-11-24\n\n ...,"[0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0]",103,"[problem, present, asthma, problem, present, n...",0,"[6942, 6905, 1440, 6942, 6905, 6021, 2844, 694..."
4,\n\nRecord date: 2094-02-16\n\nJENNIFER BOOKER...,"[0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1]",104,"[problem, present, urinary, urgency, problem, ...",0,"[6942, 6905, 8968, 8964, 6942, 6905, 8243, 292..."


In [7]:
train=data.sample(frac=0.8,random_state=200)
test=data.drop(train.index)

train.shape, test.shape

((162, 5), (40, 5))

In [8]:
trainc=datac.sample(frac=0.8,random_state=200)
testc=datac.drop(trainc.index)

trainc.shape, testc.shape

((162, 6), (40, 6))

In [9]:
from keras.preprocessing import sequence

data['lens'] = data.ind.map(lambda x: len(x))
maxlen = max(data.lens)
data = data[data.lens != 0]
maxlen

3293

In [10]:
X_test = sequence.pad_sequences(test.ind, maxlen=maxlen)
X_test = np.array([x.ravel() for x in X_test])

In [11]:
Y_test = np.array([x for x in test.diet])
Y_test

array([0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1], dtype=int64)

In [12]:
score = modelvna.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.479739284515
Test accuracy: 0.875


In [13]:
from keras.preprocessing import sequence

datac['lens'] = datac.ind.map(lambda x: len(x))
maxlenc = max(datac.lens)
datac = datac[datac.lens != 0]
maxlenc

121675

In [14]:
datac = datac[datac.lens < 10000]
maxlenc = max(datac.lens)
maxlenc

9820

In [ ]:
X_testc = sequence.pad_sequences(testc.ind, maxlen=maxlenc)
X_testc = np.array([x.ravel() for x in X_testc])
Y_testc = np.array([x for x in testc.diet])
Y_testc

array([0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1], dtype=int64)

In [ ]:
score = modelClamp.evaluate(X_testc, Y_testc, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])